## Setup the Notebook

In [2]:
import pandas as pd
import matplotlib as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

Setup Complete


## Load the Data

In [50]:
# Read the file into a variable qb_data
qb_data = pd.read_csv('FantasyPros_Fantasy_Football_Statistics_QB_2019.csv')

## Examine the Data

In [51]:
# Print the first 5 rows of the data
qb_data.head()

,Rank,Player,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT.1,YDS.1,TD.1,FL,G,FPTS,FPTS/G,OWN
0,1.0,Lamar Jackson (BAL),265.0,401.0,66.1,"3,127",7.8,36.0,6.0,23.0,176.0,"1,206",7.0,2.0,15.0,421.7,28.1,96.7%
1,2.0,Dak Prescott (DAL),388.0,596.0,65.1,"4,902",8.2,30.0,11.0,23.0,52.0,277,3.0,2.0,16.0,348.9,21.8,96.4%
2,3.0,Jameis Winston (TB),380.0,626.0,60.7,"5,109",8.2,33.0,30.0,47.0,59.0,250,1.0,5.0,16.0,335.2,21.0,89.1%
3,4.0,Russell Wilson (SEA),341.0,516.0,66.1,"4,110",8.0,31.0,5.0,48.0,75.0,342,3.0,2.0,16.0,333.5,20.8,99.3%
4,5.0,Deshaun Watson (HOU),333.0,495.0,67.3,"3,852",7.8,26.0,12.0,44.0,82.0,413,7.0,3.0,15.0,332.0,22.1,99.3%


I've checked this against the first 5 rows displayed by the source, some columns need to be renamed.

### Rename variables

In [52]:
new_names = {'CMP':'Pass_Comp',
             'ATT':'Pass_Att',
             'YDS':'Pass_Yds',
             'TD':'Pass_TD',
             'INT':'Pass_Int',
             'SACKS':'Pass_Sacks',
             'ATT.1':'Rush_Att',
             'YDS.1':'Rush_Yds',
             'TD.1':'Rush_TD',
             'FL':'Fumbles',
             'G':'Games'}
qb_data.rename(columns=new_names, inplace=True)
qb_data.columns

Index(['Rank', 'Player', 'Pass_Comp', 'Pass_Att', 'PCT', 'Pass_Yds', 'Y/A',
       'Pass_TD', 'Pass_Int', 'Pass_Sacks', 'Rush_Att', 'Rush_Yds', 'Rush_TD',
       'Fumbles', 'Games', 'FPTS', 'FPTS/G', 'OWN'],
      dtype='object')

In [53]:
qb_data.shape

(145, 18)

In [54]:
qb_data.nunique(axis=0)

Rank          143
Player        143
Pass_Comp      60
Pass_Att       63
PCT            52
Pass_Yds       67
Y/A            42
Pass_TD        26
Pass_Int       19
Pass_Sacks     33
Rush_Att       39
Rush_Yds       56
Rush_TD         7
Fumbles         8
Games          17
FPTS           69
FPTS/G         57
OWN            53
dtype: int64

In [55]:
qb_data.dtypes

Rank          float64
Player         object
Pass_Comp     float64
Pass_Att      float64
PCT           float64
Pass_Yds       object
Y/A           float64
Pass_TD       float64
Pass_Int      float64
Pass_Sacks    float64
Rush_Att      float64
Rush_Yds       object
Rush_TD       float64
Fumbles       float64
Games         float64
FPTS          float64
FPTS/G        float64
OWN            object
dtype: object

In [56]:
qb_data.describe()

,Rank,Pass_Comp,Pass_Att,PCT,Y/A,Pass_TD,Pass_Int,Pass_Sacks,Rush_Att,Rush_TD,Fumbles,Games,FPTS,FPTS/G
count,143.00000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000
mean,72.00000,79.034965,124.517483,29.390909,3.058042,5.531469,2.832168,8.888112,12.762238,0.559441,0.923077,4.517483,63.255944,5.318881
std,41.42463,129.298890,201.935698,31.482025,3.428103,9.661747,5.067135,14.251318,24.917034,1.427212,1.807952,5.812630,107.700047,7.546167
min,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.800000,-0.200000
25%,36.50000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,72.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,107.50000,97.500000,160.000000,61.900000,6.600000,6.000000,4.000000,14.500000,15.000000,0.000000,1.000000,8.000000,77.150000,11.350000
max,143.00000,408.000000,626.000000,100.000000,9.600000,36.000000,30.000000,48.000000,176.000000,9.000000,11.000000,16.000000,421.700000,28.100000


1. Pass_Yds and Rush_Yds are missing. This is likely due to a comma in the field making it a string instead of a number.
2. Minimum, 25%, and 50% are 1 or lower for all columns. This may indicate an abundace of rows with low values or values that equal 0. These rows may need to be removed for a more accurate picture.

In [57]:
# Remove the commas from Pass_Yds and Rush_Yds
qb_data['Pass_Yds'] = qb_data['Pass_Yds'].str.replace(',','')
qb_data['Pass_Yds'] = qb_data['Pass_Yds'].astype(float)
qb_data['Rush_Yds'] = qb_data['Rush_Yds'].str.replace(',','')
qb_data['Rush_Yds'] = qb_data['Rush_Yds'].astype(float)
qb_data.dtypes

Rank          float64
Player         object
Pass_Comp     float64
Pass_Att      float64
PCT           float64
Pass_Yds      float64
Y/A           float64
Pass_TD       float64
Pass_Int      float64
Pass_Sacks    float64
Rush_Att      float64
Rush_Yds      float64
Rush_TD       float64
Fumbles       float64
Games         float64
FPTS          float64
FPTS/G        float64
OWN            object
dtype: object

In [58]:
qb_data.describe()

,Rank,Pass_Comp,Pass_Att,PCT,Pass_Yds,Y/A,Pass_TD,Pass_Int,Pass_Sacks,Rush_Att,Rush_Yds,Rush_TD,Fumbles,Games,FPTS,FPTS/G
count,143.00000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000
mean,72.00000,79.034965,124.517483,29.390909,898.328671,3.058042,5.531469,2.832168,8.888112,12.762238,53.832168,0.559441,0.923077,4.517483,63.255944,5.318881
std,41.42463,129.298890,201.935698,31.482025,1491.349204,3.428103,9.661747,5.067135,14.251318,24.917034,138.689337,1.427212,1.807952,5.812630,107.700047,7.546167
min,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9.000000,0.000000,0.000000,0.000000,-0.800000,-0.200000
25%,36.50000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,72.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,107.50000,97.500000,160.000000,61.900000,1052.500000,6.600000,6.000000,4.000000,14.500000,15.000000,39.500000,0.000000,1.000000,8.000000,77.150000,11.350000
max,143.00000,408.000000,626.000000,100.000000,5109.000000,9.600000,36.000000,30.000000,48.000000,176.000000,1206.000000,9.000000,11.000000,16.000000,421.700000,28.100000


## Clean the data

### Remove redundant or unnecessary variables

In [84]:
qb_cleaned = qb_data.copy().drop(['PCT', 'Y/A', 'FPTS', 'FPTS/G','OWN'], axis=1)

### Variable Selection

In [85]:
qb_cleaned.isnull().sum()

Rank          2
Player        2
Pass_Comp     2
Pass_Att      2
Pass_Yds      2
Pass_TD       2
Pass_Int      2
Pass_Sacks    2
Rush_Att      2
Rush_Yds      2
Rush_TD       2
Fumbles       2
Games         2
dtype: int64

In [86]:
qb_cleaned[qb_cleaned.Rank.isnull()]

,Rank,Player,Pass_Comp,Pass_Att,Pass_Yds,Pass_TD,Pass_Int,Pass_Sacks,Rush_Att,Rush_Yds,Rush_TD,Fumbles,Games
143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
qb_cleaned = qb_cleaned.copy().drop([143,144], axis=0)
qb_cleaned.isnull().sum()

Rank          0
Player        0
Pass_Comp     0
Pass_Att      0
Pass_Yds      0
Pass_TD       0
Pass_Int      0
Pass_Sacks    0
Rush_Att      0
Rush_Yds      0
Rush_TD       0
Fumbles       0
Games         0
dtype: int64

### Revisiting issue found during describe()

In [88]:
qb_cleaned[qb_cleaned['Pass_Comp'] == 0].shape[0]

75

More than half of the rows in the data have a 0 in the CMP column.

In [89]:
# If a player has a 0 in every column (they didn't play), we don't want that data skewing the other data. 
qb_cleaned[(qb_cleaned['Pass_Comp'] == 0) & 
           (qb_cleaned['Pass_Att'] == 0) &
           (qb_cleaned['Pass_Yds'] == 0) &
           (qb_cleaned['Pass_TD'] == 0) &
           (qb_cleaned['Pass_Int'] == 0) &
           (qb_cleaned['Pass_Sacks'] == 0) &
           (qb_cleaned['Rush_Att'] == 0) &
           (qb_cleaned['Rush_Yds'] == 0) &
           (qb_cleaned['Rush_TD'] == 0) &
           (qb_cleaned['Fumbles'] == 0) &
           (qb_cleaned['Games'] == 0)]

,Rank,Player,Pass_Comp,Pass_Att,Pass_Yds,Pass_TD,Pass_Int,Pass_Sacks,Rush_Att,Rush_Yds,Rush_TD,Fumbles,Games
65,66.0,DeShone Kizer (LV),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,67.0,Nathan Peterman (LV),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,70.0,Davis Webb (BUF),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,72.0,Paxton Lynch (PIT),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,73.0,Jake Rudock (MIA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74,75.0,Josh Woodrum (WAS),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,76.0,Cody Kessler (NE),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,80.0,Kyle Shurmur (KC),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80,81.0,John Lovett (KC),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,82.0,Jake Dolegala (CIN),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
columns = ['Pass_Comp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'Pass_Sacks', 
           'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Fumbles','Games']
qb_cleaned = qb_cleaned.replace(0, pd.np.nan).dropna(axis=0, how='all', subset=columns).fillna(0)
qb_cleaned.sample(10)

,Rank,Player,Pass_Comp,Pass_Att,Pass_Yds,Pass_TD,Pass_Int,Pass_Sacks,Rush_Att,Rush_Yds,Rush_TD,Fumbles,Games
24,25.0,Andy Dalton (CIN),314.0,528.0,3494.0,16.0,14.0,37.0,32.0,73.0,4.0,4.0,13.0
51,52.0,Cam Newton (CAR),50.0,89.0,572.0,0.0,1.0,6.0,5.0,-2.0,0.0,2.0,2.0
8,9.0,Carson Wentz (PHI),388.0,607.0,4039.0,27.0,7.0,37.0,62.0,243.0,1.0,7.0,16.0
64,65.0,Trevor Siemian (NYJ),3.0,6.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
44,45.0,Matt Schaub (ATL),50.0,67.0,580.0,3.0,1.0,2.0,3.0,-3.0,0.0,1.0,7.0
14,15.0,Philip Rivers (FA),390.0,591.0,4615.0,23.0,20.0,34.0,12.0,29.0,0.0,3.0,16.0
63,64.0,Trace McSorley (BAL),0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
31,32.0,Teddy Bridgewater (NO),133.0,196.0,1384.0,9.0,2.0,12.0,28.0,31.0,0.0,0.0,10.0
37,38.0,Jeff Driskel (DET),62.0,105.0,685.0,4.0,4.0,11.0,22.0,151.0,1.0,0.0,4.0
141,142.0,Jarrett Stidham (NE),2.0,4.0,14.0,0.0,1.0,1.0,2.0,-2.0,0.0,0.0,3.0


In [105]:
qb_cleaned.shape[0]

89

In 'Player' there is text behind the player's name to indicate what team he plays for. This is the current team, for example Philip Rivers was released from the Chargers at the end of the 2019 season. It would make sense that the letters behind his name should read '(SD)' instead of '(FA)'. Since this analysis is of the player, we will remove the team name from the 'Player' variable. 

In [116]:
qb_cleaned['Player'] = qb_cleaned['Player'].str.replace(pat=r'\(.*\)', repl='')
qb_cleaned.sample(5)

,Rank,Player,Pass_Comp,Pass_Att,Pass_Yds,Pass_TD,Pass_Int,Pass_Sacks,Rush_Att,Rush_Yds,Rush_TD,Fumbles,Games
30,31.0,Case Keenum,160.0,247.0,1707.0,11.0,5.0,15.0,9.0,12.0,1.0,3.0,10.0
44,45.0,Matt Schaub,50.0,67.0,580.0,3.0,1.0,2.0,3.0,-3.0,0.0,1.0,7.0
4,5.0,Deshaun Watson,333.0,495.0,3852.0,26.0,12.0,44.0,82.0,413.0,7.0,3.0,15.0
62,63.0,Sean Mannion,12.0,21.0,126.0,0.0,2.0,0.0,6.0,-5.0,0.0,1.0,3.0
98,99.0,Joe Webb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
